# JsonOutputParser

- 著者: [Jaehun Choi](https://github.com/ash-hun)
- ピアレビュー: [Jeongeun Lim](https://www.linkedin.com/in/jeongeun-lim-808978188/), [frimer](https://github.com/brian604)
- 校正: [BokyungisaGod](https://github.com/BokyungisaGod)
- これは [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial) の一部です

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/04-JsonOutputParser.ipynb)
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/03-OutputParser/04-JsonOutputParser.ipynb)

## 概要

このチュートリアルでは、```JsonOutputParser```の実装について説明します。
```JsonOutputParser```は、ユーザーが希望するJSONスキーマを指定できるツールです。これは、LLM(大規模言語モデル)がデータをクエリし、指定されたスキーマに準拠したJSON形式で結果を返すことを可能にするように設計されています。
LLMがデータを正確かつ効率的に処理し、希望する形式でJSONを生成するためには、モデルが十分な容量(例: 知能)を持っている必要があります。例えば、llama-70Bモデルはllama-8Bモデルと比較してより大きな容量を持っており、複雑なデータを処理するのにより適しています。

**[注意]**

**JSON (JavaScript Object Notation)**は、データの保存と構造化に使用される軽量なデータ交換形式です。Web開発において重要な役割を果たし、サーバーとクライアント間の通信に広く使用されています。JSONは、読みやすく、機械が解析および生成しやすいテキストに基づいています。

JSONデータはキー/値のペアで構成されます。ここで、「キー」は文字列で、「値」はさまざまなデータ型にできます。JSONには2つの主要な構造があります:

- オブジェクト: 中括弧 { } で囲まれたキー/値のペアのコレクション。各キーはコロン ( : ) を使用してその値に関連付けられ、複数のキー/値のペアはカンマ ( , ) で区切られます。
- 配列: 角括弧 [ ] で囲まれた値の順序付きリスト。配列内の値はカンマ ( , ) で区切られます。

```json
{
  "name": "太郎",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "メインストリート123",
    "city": "東京"
  }
}
```

### 目次

- [概要](#概要)
- [環境設定](#環境設定)
- [PydanticでJsonOutputParserを使用](#PydanticでJsonOutputParserを使用)
- [PydanticなしでJsonOutputParserを使用](#PydanticなしでJsonOutputParserを使用)

### 参考文献

- [JsonOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html)
----

## 環境設定

環境をセットアップします。詳細については[環境設定](https://wikidocs.net/257836)を参照してください。

**[注意]**
- ```langchain-opentutorial```は、チュートリアル用の使いやすい環境設定、便利な関数、ユーティリティを提供するパッケージです。
- 詳細については[```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi)をご確認ください。

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# 必要なパッケージをインストール
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [3]:
# 環境変数を設定
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "04-JsonOutputParser",
    }
)

環境変数が正常に設定されました。


あるいは、```.env```ファイルに```OPENAI_API_KEY```を設定して読み込むこともできます。
**[注意]** 前の手順で```OPENAI_API_KEY```を既に設定している場合、これは不要です。

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## ```Pydantic```で```JsonOutputParser```を使用

JSON形式で出力を生成する必要がある場合、LangChainの```JsonOutputParser```を使用して簡単に実装できます。JSON形式で出力を生成する方法は2つあります:

- ```Pydantic```を使用
- ```Pydantic```を使用しない

以下の手順に従って実装してください:

まず、必要なモジュールをインポートします。

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [6]:
# OpenAIオブジェクトを作成
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

出力データのスキーマ形式を定義します。

In [7]:
# Pydanticを使用して出力形式のデータスキーマを定義
class Topic(BaseModel):
    description: str = Field(description="トピックの簡潔な説明")
    hashtags: str = Field(description="ハッシュタグ形式のキーワード（最低2つ）")

```JsonOutputParser```を使用してパーサーを設定し、プロンプトテンプレートに指示を注入します。

In [8]:
# 質問を記述
question = "地球温暖化の深刻さについて説明してください。"

# パーサーを設定し、プロンプトテンプレートに指示を注入
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

出力は、以下のJSONスキーマに準拠したJSONインスタンスとしてフォーマットする必要があります。

例として、スキーマ {"properties": {"foo": {"title": "Foo", "description": "文字列のリスト", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
の場合、オブジェクト {"foo": ["bar", "baz"]} はスキーマの正しくフォーマットされたインスタンスです。オブジェクト {"properties": {"foo": ["bar", "baz"]}} は正しくフォーマットされていません。

出力スキーマは次のとおりです:
```
{"properties": {"description": {"description": "トピックの簡潔な説明", "title": "Description", "type": "string"}, "hashtags": {"description": "ハッシュタグ形式のキーワード（最低2つ)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [9]:
# プロンプトテンプレートを設定
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは親切なAIアシスタントです。質問に簡潔に答えてください。"),
        ("user", "#フォーマット: {format_instructions}\n\n#質問: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# プロンプト、モデル、JsonOutputParserをチェーンに結合
chain = prompt | model | parser

# 質問でチェーンを実行
answer = chain.invoke({"question": question})

In [10]:
# 型を確認
type(answer)

dict

In [11]:
# 回答オブジェクトを出力
answer

{'description': '地球温暖化は、温室効果ガスのレベル上昇により地球の平均表面温度が上昇することを特徴とする重大な環境問題です。これは深刻な気象変化、海面上昇、生物多様性と人間の生活への影響をもたらします。',
 'hashtags': '#地球温暖化 #気候変動 #環境影響'}

## ```Pydantic```なしで```JsonOutputParser```を使用

```Pydantic```を使用せずにJSON形式で出力を生成できます。

以下の手順に従って実装してください:

In [12]:
# 質問を記述
question = "地球温暖化に関する情報を提供してください。説明はdescriptionに、関連キーワードは`hashtags`に含めてください。"

# JsonOutputParserを初期化
parser = JsonOutputParser()

# プロンプトテンプレートを設定
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは親切なAIアシスタントです。質問に簡潔に答えてください。"),
        ("user", "#フォーマット: {format_instructions}\n\n#質問: {question}"),
    ]
)

# プロンプトに指示を注入
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# プロンプト、モデル、JsonOutputParserをチェーンに結合
chain = prompt | model | parser

# 質問でチェーンを実行
response = chain.invoke({"question": question})
print(response)

{'description': '地球温暖化とは、主に人間の活動による地球の平均表面温度の長期的な上昇を指します。これは主に、化石燃料の燃焼、森林伐採、産業プロセスから生じる二酸化炭素、メタン、亜酸化窒素などの温室効果ガスの排出によるものです。これらの排出は、大気中に熱が閉じ込められる温室効果をもたらします。この温暖化は気象パターン、海面、生態系に重大な影響を及ぼし、気候変動に寄与し、生物多様性と人間社会にリスクをもたらします。', 'hashtags': ['#地球温暖化', '#気候変動', '#温室効果ガス', '#炭素排出', '#化石燃料', '#森林伐採', '#持続可能性', '#環境影響']}
